In [6]:
# Define Neo4j connections
import pandas as pd
from neo4j import GraphDatabase
host = 'bolt://3.235.89.19:7687'
user = 'neo4j'
password = 'ballast-set-proficiencies'
driver = GraphDatabase.driver(host,auth=(user, password))

def run_query(query, params={}):
    with driver.session() as session:
        result = session.run(query, params)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())

In [5]:
!pip install neo4j

     |████████████████████████████████| 89 kB 5.1 MB/s  eta 0:00:01
  Created wheel for neo4j: filename=neo4j-4.4.2-py3-none-any.whl size=115363 sha256=52993c8ab4b1012283172d91c46516dad47bd804fee00947192ec3dd36e53447
  Stored in directory: /home/kangjunekoo/.cache/pip/wheels/10/d6/28/95029d7f69690dbc3b93e4933197357987de34fbd44b50a0e4
Successfully built neo4j


In [7]:
run_query("""
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/mathbeveridge/gameofthrones/master/data/got-s1-edges.csv" as row
MERGE (s:Character{name:row.Source})
MERGE (t:Character{name:row.Target})
MERGE (s)-[i:INTERACTS]-(t)
SET i.weight = toInteger(row.Weight)
""")

ServiceUnavailable: Couldn't connect to 3.235.89.19:7687 (resolved to ('3.235.89.19:7687',)):
Timed out trying to establish connection to ResolvedIPv4Address(('3.235.89.19', 7687))

In [ ]:
run_query("""
CALL gds.graph.create('gots1', 'Character', {INTERACTS:{orientation:'UNDIRECTED', properties:'weight'}})
""")

OSError: libtorch_cuda_cpp.so: cannot open shared object file: No such file or directory

In [ ]:
run_query("""
CALL gds.louvain.write('gots1', {writeProperty:'louvain', relationshipWeightProperty:'weight'})
""")

In [ ]:
run_query("""
CALL gds.fastRP.write('gots1', {writeProperty:'embedding', embeddingDimension:56, relationshipWeightProperty:'weight'})
""")

In [8]:
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
import seaborn as sns


def tsne(embeddings, hue=None):
    tsne = TSNE(n_components=2, n_iter=300)
    tsne_results = tsne.fit_transform(embeddings['embedding'].to_list())

    embeddings['tsne_x'] = [x[0] for x in list(tsne_results)]
    embeddings['tsne_y'] = [x[1] for x in list(tsne_results)]

    plt.figure(figsize=(16,10))
    sns.scatterplot(
        x="tsne_x", y="tsne_y",
        hue=hue,
        palette="deep",
        data=embeddings,
        legend="full",
        alpha=0.9
    )
    # Add captions
    #for i in range(embeddings.shape[0]):
    #    plt.text(x=embeddings['tsne_x'][i]+0.3,y=embeddings['tsne_y'][i]+0.3,s=embeddings.character[i], 
    #          fontdict=dict(color='black',size=10),)